# Conteo 3 bolillos por hectárea

In [2]:
import cv2
import numpy as np

# Cargar imagen
image = cv2.imread("/Users/samanthabritoozuna/Desktop/Opti_reto/Fotos/Color.jpg")
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Máscara para magenta (ajustar rangos HSV)
lower_magenta = np.array([140, 50, 50])
upper_magenta = np.array([170, 255, 255])
mask_magenta = cv2.inRange(hsv, lower_magenta, upper_magenta)

# Encontrar contornos
contours, _ = cv2.findContours(mask_magenta, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(f"Número de polígonos detectados: {len(contours)}")

# Parámetros de escala y espaciamiento
scale_factor = 100 / .6  # píxeles por metro
#espaciado_plantas = 0.316  # en metros
espaciado_plantas_cm = 3.5 # cm
espaciado_plantas = espaciado_plantas_cm / 100  # en metros


# Imagen 1: contornos numerados
img_contornos = image.copy()
for i, cnt in enumerate(contours):
    cv2.drawContours(img_contornos, [cnt], 0, (0, 255, 0), 2)
    M = cv2.moments(cnt)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        cv2.putText(img_contornos, f"{i+1}", (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

cv2.imwrite("contornos_numerados_2.jpg", img_contornos)

# Imagen 2: grid 3 bolillos
img_grid = image.copy()

for i, cnt in enumerate(contours):
    # Área
    area_px = cv2.contourArea(cnt)
    area_m2 = area_px / (scale_factor ** 2)
    area_ha = area_m2 / 10_000

    # Máscara del polígono
    mask_poly = np.zeros_like(mask_magenta)
    cv2.drawContours(mask_poly, [cnt], 0, 255, -1)

    # Espaciado en píxeles
    spacing_px = int(espaciado_plantas * scale_factor)
    '''
    spacing_px = int(espaciado_plantas * scale_factor)
    if spacing_px == 0:
      spacing_px = 1 
      '''
    height, width = mask_poly.shape

    # Grid 3 bolillos (hexagonal alternado)
    plantas = []
    for y in range(0, height, spacing_px):
        offset = (spacing_px // 2) if (y // spacing_px) % 2 == 1 else 0
        for x in range(0 + offset, width, spacing_px):
            if mask_poly[y, x] == 255:
                plantas.append((x, y))
                cv2.circle(img_grid, (x, y), 2, (255, 0, 0), -1)

    print(f"Polígono {i+1}: Área = {area_m2:.2f} m² ({area_ha:.4f} ha), Plantas = {len(plantas)}")

cv2.imwrite("grid_3_bolillos_3.jpg", img_grid)


Número de polígonos detectados: 31
Polígono 1: Área = 0.64 m² (0.0001 ha), Plantas = 701
Polígono 2: Área = 0.99 m² (0.0001 ha), Plantas = 1104
Polígono 3: Área = 1.01 m² (0.0001 ha), Plantas = 1128
Polígono 4: Área = 1.04 m² (0.0001 ha), Plantas = 1176
Polígono 5: Área = 0.90 m² (0.0001 ha), Plantas = 1003
Polígono 6: Área = 0.51 m² (0.0001 ha), Plantas = 577
Polígono 7: Área = 0.18 m² (0.0000 ha), Plantas = 211
Polígono 8: Área = 0.99 m² (0.0001 ha), Plantas = 1111
Polígono 9: Área = 1.01 m² (0.0001 ha), Plantas = 1128
Polígono 10: Área = 1.05 m² (0.0001 ha), Plantas = 1176
Polígono 11: Área = 0.90 m² (0.0001 ha), Plantas = 1003
Polígono 12: Área = 0.75 m² (0.0001 ha), Plantas = 849
Polígono 13: Área = 0.68 m² (0.0001 ha), Plantas = 752
Polígono 14: Área = 1.02 m² (0.0001 ha), Plantas = 1140
Polígono 15: Área = 0.88 m² (0.0001 ha), Plantas = 992
Polígono 16: Área = 0.77 m² (0.0001 ha), Plantas = 862
Polígono 17: Área = 0.67 m² (0.0001 ha), Plantas = 756
Polígono 18: Área = 0.74 m² (0

True

# Distancias

In [42]:
# 1. Detectar centro de la región verde (camioneta)

image = cv2.imread("Color.jpg")
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

green_rgb = np.uint8([[[111, 237, 75]]])
green_hsv = cv2.cvtColor(green_rgb, cv2.COLOR_RGB2HSV)[0][0]
lower_green = np.array([green_hsv[0]-10, 50, 50])
upper_green = np.array([green_hsv[0]+10, 255, 255])
mask_green = cv2.inRange(hsv, lower_green, upper_green)



# Encontrar el centroide de la zona verde
contours_green, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if contours_green:
    M_green = cv2.moments(contours_green[0])
    cx_green = int(M_green["m10"] / M_green["m00"])
    cy_green = int(M_green["m01"] / M_green["m00"])
    punto_salida = (cx_green, cy_green)
else:
    raise Exception("No se detectó la región verde (camioneta)")

# 2. Cálculo de distancias y tiempos
print("\n--- MATRIZ DE DISTANCIAS ---")
vel_ida_kmh = 25
vel_regreso_kmh = 40
vel_ida_mps = vel_ida_kmh * 1000 / 3600
vel_regreso_mps = vel_regreso_kmh * 1000 / 3600

for i, cnt in enumerate(contours):
    # Obtener el vértice más cercano al punto de salida
    dist_min = float("inf")
    punto_objetivo = None
    for pt in cnt:
        x, y = pt[0]
        dist = np.hypot(x - punto_salida[0], y - punto_salida[1])
        if dist < dist_min:
            dist_min = dist
            punto_objetivo = (x, y)

    # Convertir distancia a metros
    dist_metros = dist_min / scale_factor

    # Tiempo de ida y regreso
    tiempo_ida = dist_metros / vel_ida_mps
    tiempo_regreso = dist_metros / vel_regreso_mps
    tiempo_total = tiempo_ida + tiempo_regreso

    print(f"Polígono {i+1}: Distancia = {dist_metros:.2f} m | Tiempo ida = {tiempo_ida:.1f}s | regreso = {tiempo_regreso:.1f}s | total = {tiempo_total:.1f}s")



--- MATRIZ DE DISTANCIAS ---
Polígono 1: Distancia = 5.58 m | Tiempo ida = 0.8s | regreso = 0.5s | total = 1.3s
Polígono 2: Distancia = 4.06 m | Tiempo ida = 0.6s | regreso = 0.4s | total = 1.0s
Polígono 3: Distancia = 4.06 m | Tiempo ida = 0.6s | regreso = 0.4s | total = 1.0s
Polígono 4: Distancia = 4.15 m | Tiempo ida = 0.6s | regreso = 0.4s | total = 1.0s
Polígono 5: Distancia = 4.38 m | Tiempo ida = 0.6s | regreso = 0.4s | total = 1.0s
Polígono 6: Distancia = 4.69 m | Tiempo ida = 0.7s | regreso = 0.4s | total = 1.1s
Polígono 7: Distancia = 2.65 m | Tiempo ida = 0.4s | regreso = 0.2s | total = 0.6s
Polígono 8: Distancia = 2.56 m | Tiempo ida = 0.4s | regreso = 0.2s | total = 0.6s
Polígono 9: Distancia = 2.57 m | Tiempo ida = 0.4s | regreso = 0.2s | total = 0.6s
Polígono 10: Distancia = 2.71 m | Tiempo ida = 0.4s | regreso = 0.2s | total = 0.6s
Polígono 11: Distancia = 3.04 m | Tiempo ida = 0.4s | regreso = 0.3s | total = 0.7s
Polígono 12: Distancia = 3.60 m | Tiempo ida = 0.5s | r

In [43]:
import csv

# Crear y escribir archivo CSV
with open("distancias_camioneta.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Poligono", "Vertice_X", "Vertice_Y", "Distancia_m", "Tiempo_ida_s", "Tiempo_regreso_s", "Tiempo_total_s"])

    for i, cnt in enumerate(contours):
        # Buscar el vértice más cercano
        dist_min = float("inf")
        punto_objetivo = None
        for pt in cnt:
            x, y = pt[0]
            dist = np.hypot(x - punto_salida[0], y - punto_salida[1])
            if dist < dist_min:
                dist_min = dist
                punto_objetivo = (x, y)

        # Cálculos
        dist_metros = dist_min / scale_factor
        tiempo_ida = dist_metros / vel_ida_mps
        tiempo_regreso = dist_metros / vel_regreso_mps
        tiempo_total = tiempo_ida + tiempo_regreso

        # Escribir al archivo
        writer.writerow([
            i + 1,
            punto_objetivo[0],
            punto_objetivo[1],
            round(dist_metros, 2),
            round(tiempo_ida, 1),
            round(tiempo_regreso, 1),
            round(tiempo_total, 1)
        ])


In [44]:
image = cv2.imread("Color.jpg")
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

green_rgb = np.uint8([[[111, 237, 75]]])
green_hsv = cv2.cvtColor(green_rgb, cv2.COLOR_RGB2HSV)[0][0]
lower_green = np.array([green_hsv[0]-10, 50, 50])
upper_green = np.array([green_hsv[0]+10, 255, 255])
mask_green = cv2.inRange(hsv, lower_green, upper_green)

cv2.imwrite("mascara_verde.jpg", mask_green)  # Guardar para revisar

contours_green, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if contours_green:
    M_green = cv2.moments(contours_green[0])
    cx_green = int(M_green["m10"] / M_green["m00"])
    cy_green = int(M_green["m01"] / M_green["m00"])
    punto_salida = (cx_green, cy_green)
    print(f"Camioneta detectada en: {punto_salida}")
else:
    raise Exception("No se detectó la región verde (camioneta)")

# --- Calcular distancias y guardar CSV
vel_ida_kmh = 25
vel_regreso_kmh = 40
vel_ida_mps = vel_ida_kmh * 1000 / 3600
vel_regreso_mps = vel_regreso_kmh * 1000 / 3600

with open("distancias_camioneta.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Poligono", "Vertice_X", "Vertice_Y", "Distancia_m", "Tiempo_ida_s", "Tiempo_regreso_s", "Tiempo_total_s"])

    for i, cnt in enumerate(contours):
        dist_min = float("inf")
        punto_objetivo = None
        for pt in cnt:
            x, y = pt[0]
            dist = np.hypot(x - punto_salida[0], y - punto_salida[1])
            if dist < dist_min:
                dist_min = dist
                punto_objetivo = (x, y)

        dist_metros = dist_min / scale_factor
        tiempo_ida = dist_metros / vel_ida_mps
        tiempo_regreso = dist_metros / vel_regreso_mps
        tiempo_total = tiempo_ida + tiempo_regreso

        print(f"Polígono {i+1}: Dist = {dist_metros:.2f} m | Tiempo total = {tiempo_total:.1f} s")

        writer.writerow([
            i + 1,
            punto_objetivo[0],
            punto_objetivo[1],
            round(dist_metros, 2),
            round(tiempo_ida, 1),
            round(tiempo_regreso, 1),
            round(tiempo_total, 1)
        ])

Camioneta detectada en: (1197, 535)
Polígono 1: Dist = 5.58 m | Tiempo total = 1.3 s
Polígono 2: Dist = 4.06 m | Tiempo total = 1.0 s
Polígono 3: Dist = 4.06 m | Tiempo total = 1.0 s
Polígono 4: Dist = 4.15 m | Tiempo total = 1.0 s
Polígono 5: Dist = 4.38 m | Tiempo total = 1.0 s
Polígono 6: Dist = 4.69 m | Tiempo total = 1.1 s
Polígono 7: Dist = 2.65 m | Tiempo total = 0.6 s
Polígono 8: Dist = 2.56 m | Tiempo total = 0.6 s
Polígono 9: Dist = 2.57 m | Tiempo total = 0.6 s
Polígono 10: Dist = 2.71 m | Tiempo total = 0.6 s
Polígono 11: Dist = 3.04 m | Tiempo total = 0.7 s
Polígono 12: Dist = 3.60 m | Tiempo total = 0.8 s
Polígono 13: Dist = 1.76 m | Tiempo total = 0.4 s
Polígono 14: Dist = 2.37 m | Tiempo total = 0.6 s
Polígono 15: Dist = 1.41 m | Tiempo total = 0.3 s
Polígono 16: Dist = 1.38 m | Tiempo total = 0.3 s
Polígono 17: Dist = 1.80 m | Tiempo total = 0.4 s
Polígono 18: Dist = 0.76 m | Tiempo total = 0.2 s
Polígono 19: Dist = 2.33 m | Tiempo total = 0.5 s
Polígono 20: Dist = 0.0

In [45]:
import cv2
import numpy as np
import networkx as nx
import csv

# === PARÁMETROS ===
scale_factor = 100 / 0.006  # píxeles por metro
vel_ida = 25 * 1000 / 3600  # m/s
vel_regreso = 40 * 1000 / 3600  # m/s

# === CARGAR IMAGEN Y CONVERTIR A HSV ===
image = cv2.imread("Color.jpg")
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# === DETECTAR CONTORNOS MAGENTA ===
lower_magenta = np.array([140, 50, 50])
upper_magenta = np.array([170, 255, 255])
mask_magenta = cv2.inRange(hsv, lower_magenta, upper_magenta)
contours, _ = cv2.findContours(mask_magenta, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# === DETECTAR ZONA VERDE (CAMIONETA) ===
lower_green = np.array([40, 100, 100])
upper_green = np.array([80, 255, 255])
mask_green = cv2.inRange(hsv, lower_green, upper_green)
contours_green, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if not contours_green:
    raise Exception("No se detectó la región verde (camioneta)")

M_green = cv2.moments(contours_green[0])
cx_green = int(M_green["m10"] / M_green["m00"])
cy_green = int(M_green["m01"] / M_green["m00"])
punto_salida = (cx_green, cy_green)

# === CONSTRUIR GRAFO DE CAMINOS (bordes amarillos) ===
G = nx.Graph()
poligono_vertices = []  # Para guardar los vértices por polígono

for i, cnt in enumerate(contours):
    vertices = []
    for j in range(len(cnt)):
        p1 = tuple(cnt[j][0])
        p2 = tuple(cnt[(j + 1) % len(cnt)][0])
        dist = np.linalg.norm(np.array(p1) - np.array(p2))
        G.add_edge(p1, p2, weight=dist / scale_factor)  # distancia en metros
        vertices.append(p1)
    poligono_vertices.append(vertices)

# === CONECTAR ALMACÉN AL GRAFO (nodo más cercano) ===
all_nodes = list(G.nodes())
distancias_almacen = [np.linalg.norm(np.array(punto_salida) - np.array(n)) for n in all_nodes]
idx_min = np.argmin(distancias_almacen)
nodo_mas_cercano = all_nodes[idx_min]
G.add_edge(punto_salida, nodo_mas_cercano, weight=distancias_almacen[idx_min] / scale_factor)

# === CALCULAR RUTAS DESDE ALMACÉN Y GUARDAR CSV ===
with open("rutas_desde_camioneta.csv", mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Poligono", "Distancia_ida_m", "Tiempo_ida_s", "Tiempo_regreso_s", "Tiempo_total_s"])

    for i, vertices in enumerate(poligono_vertices):
        # Escoger el vértice más cercano del polígono al almacén
        distancias = [nx.shortest_path_length(G, source=punto_salida, target=v, weight="weight") for v in vertices]
        idx_v = np.argmin(distancias)
        nodo_objetivo = vertices[idx_v]
        distancia = distancias[idx_v]
        tiempo_ida = distancia / vel_ida
        tiempo_regreso = distancia / vel_regreso
        writer.writerow([i + 1, round(distancia, 2), round(tiempo_ida, 1), round(tiempo_regreso, 1), round(tiempo_ida + tiempo_regreso, 1)])

# === MATRIZ DE DISTANCIAS ENTRE POLÍGONOS ===
n = len(poligono_vertices)
matriz = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        if i == j:
            continue
        min_dist = float("inf")
        for v1 in poligono_vertices[i]:
            for v2 in poligono_vertices[j]:
                try:
                    d = nx.shortest_path_length(G, source=v1, target=v2, weight="weight")
                    if d < min_dist:
                        min_dist = d
                except nx.NetworkXNoPath:
                    continue
        matriz[i, j] = min_dist

# Guardar matriz en CSV
with open("matriz_distancias_poligonos.csv", "w", newline="") as f:
    writer = csv.writer(f)
    header = [""] + [f"P{j+1}" for j in range(n)]
    writer.writerow(header)
    for i in range(n):
        row = [f"P{i+1}"] + [round(matriz[i, j], 2) if i != j else 0 for j in range(n)]
        writer.writerow(row)

# === VISUALIZAR LAS RUTAS ===
img_rutas = image.copy()
for i, vertices in enumerate(poligono_vertices):
    distancias = [nx.shortest_path_length(G, source=punto_salida, target=v, weight="weight") for v in vertices]
    idx_v = np.argmin(distancias)
    nodo_objetivo = vertices[idx_v]
    ruta = nx.shortest_path(G, source=punto_salida, target=nodo_objetivo, weight="weight")
    for k in range(len(ruta) - 1):
        cv2.line(img_rutas, ruta[k], ruta[k+1], (0, 0, 255), 2)
cv2.imwrite("rutas_camioneta.jpg", img_rutas)


NetworkXNoPath: Node (872, 1461) not reachable from (1197, 535)

In [50]:
import cv2
import numpy as np
import networkx as nx
import csv

# === PARÁMETROS ===
scale_factor = 100 / 0.006  # píxeles por metro
vel_ida = 25 * 1000 / 3600  # m/s
vel_regreso = 40 * 1000 / 3600  # m/s
conexion_max_px = 3500  # máxima distancia en píxeles para conectar nodos


# === CARGAR IMAGEN Y CONVERTIR A HSV ===
image = cv2.imread("Color.jpg")
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

In [51]:


# === DETECTAR CONTORNOS MAGENTA (POLÍGONOS) ===
lower_magenta = np.array([140, 50, 50])
upper_magenta = np.array([170, 255, 255])
mask_magenta = cv2.inRange(hsv, lower_magenta, upper_magenta)
contours_magenta, _ = cv2.findContours(mask_magenta, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# === DETECTAR LÍNEAS AMARILLAS (CAMINOS) ===
lower_yellow = np.array([20, 100, 100])
upper_yellow = np.array([40, 255, 255])
mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

# Procesamiento de líneas amarillas
kernel = np.ones((3,3), np.uint8)
mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_OPEN, kernel)
mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_CLOSE, kernel)
contours_yellow, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# === DETECTAR ZONA VERDE (CAMIONETA) ===
lower_green = np.array([40, 100, 100])
upper_green = np.array([80, 255, 255])
mask_green = cv2.inRange(hsv, lower_green, upper_green)
contours_green, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if not contours_green:
    raise Exception("No se detectó la región verde (camioneta)")


M_green = cv2.moments(contours_green[0])
cx_green = int(M_green["m10"] / M_green["m00"])
cy_green = int(M_green["m01"] / M_green["m00"])
punto_salida = (cx_green, cy_green)

# === CONSTRUIR GRAFO DE CAMINOS ===
G = nx.Graph()
poligono_vertices = []

# 1. Añadir bordes de los polígonos magenta
for i, cnt in enumerate(contours_magenta):
    vertices = []
    for j in range(len(cnt)):
        p1 = tuple(cnt[j][0])
        p2 = tuple(cnt[(j + 1) % len(cnt)][0])
        dist = np.linalg.norm(np.array(p1) - np.array(p2))
        G.add_edge(p1, p2, weight=dist / scale_factor)
        vertices.append(p1)
    poligono_vertices.append(vertices)

# 2. Añadir líneas amarillas como caminos
for cnt in contours_yellow:
    epsilon = 0.005 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    
    for j in range(len(approx) - 1):
        p1 = tuple(approx[j][0])
        p2 = tuple(approx[j+1][0])
        dist = np.linalg.norm(np.array(p1) - np.array(p2))
        if dist > 0:
            G.add_edge(p1, p2, weight=dist / scale_factor)

# 3. Conectar polígonos con líneas amarillas cercanas
for poly_vertices in poligono_vertices:
    for v in poly_vertices:
        min_dist = float('inf')
        closest_yellow = None
        for cnt in contours_yellow:
            for point in cnt:
                p = tuple(point[0])
                dist = np.linalg.norm(np.array(v) - np.array(p))
                if dist < min_dist and dist < conexion_max_px:
                    min_dist = dist
                    closest_yellow = p
        
        if closest_yellow:
            G.add_edge(v, closest_yellow, weight=min_dist / scale_factor)

# 4. Conectar camioneta a líneas amarillas cercanas
yellow_points = []
for cnt in contours_yellow:
    for point in cnt:
        yellow_points.append(tuple(point[0]))

for point in yellow_points:
    dist = np.linalg.norm(np.array(punto_salida) - np.array(point))
    if dist < conexion_max_px:
        G.add_edge(punto_salida, point, weight=dist / scale_factor)

# === CALCULAR RUTAS ===
with open("rutas_desde_camioneta.csv", mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Poligono", "Distancia_ida_m", "Tiempo_ida_s", "Tiempo_regreso_s", "Tiempo_total_s"])

    for i, vertices in enumerate(poligono_vertices):
        min_dist = float("inf")
        nodo_objetivo = None
        for v in vertices:
            try:
                d = nx.shortest_path_length(G, source=punto_salida, target=v, weight="weight")
                if d < min_dist:
                    min_dist = d
                    nodo_objetivo = v
            except nx.NetworkXNoPath:
                continue
        
        if nodo_objetivo is None:
            print(f"⚠️ No se pudo encontrar ruta al polígono {i+1}")
            writer.writerow([i + 1, "N/A", "N/A", "N/A", "N/A"])
        else:
            tiempo_ida = min_dist / vel_ida
            tiempo_regreso = min_dist / vel_regreso
            writer.writerow([
                i + 1, 
                round(min_dist, 2), 
                round(tiempo_ida, 1), 
                round(tiempo_regreso, 1), 
                round(tiempo_ida + tiempo_regreso, 1)
            ])

# === VISUALIZACIÓN ===
img_rutas = image.copy()
for i, vertices in enumerate(poligono_vertices):
    min_dist = float("inf")
    nodo_objetivo = None
    for v in vertices:
        try:
            d = nx.shortest_path_length(G, source=punto_salida, target=v, weight="weight")
            if d < min_dist:
                min_dist = d
                nodo_objetivo = v
        except nx.NetworkXNoPath:
            continue
    
    if nodo_objetivo:
        ruta = nx.shortest_path(G, source=punto_salida, target=nodo_objetivo, weight="weight")
        for k in range(len(ruta) - 1):
            cv2.line(img_rutas, ruta[k], ruta[k+1], (0, 0, 255), 2)

cv2.imwrite("rutas_camioneta.jpg", img_rutas)
print("Proceso completado correctamente")

Proceso completado correctamente


In [52]:

# === DETECTAR LÍNEAS AMARILLAS (ÚNICOS CAMINOS VÁLIDOS) ===
lower_yellow = np.array([20, 100, 100])
upper_yellow = np.array([40, 255, 255])
mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

# Mejorar conexión de líneas amarillas
kernel = np.ones((3,3), np.uint8)
mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_CLOSE, kernel)  # Cierra huecos
contours_yellow, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# === DETECTAR PUNTO DE SALIDA (VERDE) ===
lower_green = np.array([40, 100, 100])
upper_green = np.array([80, 255, 255])
mask_green = cv2.inRange(hsv, lower_green, upper_green)
contours_green, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if not contours_green:
    raise Exception("No se detectó la región verde (camioneta)")

M_green = cv2.moments(contours_green[0])
cx_green = int(M_green["m10"] / M_green["m00"])
cy_green = int(M_green["m01"] / M_green["m00"])
punto_salida = (cx_green, cy_green)

# === CONSTRUIR GRAFO SOLO CON LÍNEAS AMARILLAS ===
G = nx.Graph()

# Añadir todas las líneas amarillas como aristas
for cnt in contours_yellow:
    epsilon = 0.01 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)  # Simplificar contorno
    for j in range(len(approx) - 1):
        p1 = tuple(approx[j][0])
        p2 = tuple(approx[j+1][0])
        dist = np.linalg.norm(np.array(p1) - np.array(p2))
        G.add_edge(p1, p2, weight=dist / scale_factor)

# === CONECTAR PUNTO DE SALIDA A LA RED AMARILLA ===
yellow_nodes = list(G.nodes())
min_dist = float('inf')
punto_conexion = None

for node in yellow_nodes:
    dist = np.linalg.norm(np.array(punto_salida) - np.array(node))
    if dist < conexion_max_px and dist < min_dist:
        min_dist = dist
        punto_conexion = node

if punto_conexion:
    G.add_edge(punto_salida, punto_conexion, weight=min_dist / scale_factor)
else:
    raise Exception("El punto de salida no está conectado a ninguna línea amarilla")

# === DETECTAR POLÍGONOS (MAGENTA) Y CONECTARLOS A LÍNEAS AMARILLAS ===
lower_magenta = np.array([140, 50, 50])
upper_magenta = np.array([170, 255, 255])
mask_magenta = cv2.inRange(hsv, lower_magenta, upper_magenta)
contours_magenta, _ = cv2.findContours(mask_magenta, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

poligono_vertices = []
for i, cnt in enumerate(contours_magenta):
    # Encontrar el punto más cercano en el polígono a la red amarilla
    min_dist = float('inf')
    punto_poligono = None
    punto_amarillo = None
    
    for point in cnt[:,0,:]:
        for node in yellow_nodes:
            dist = np.linalg.norm(point - np.array(node))
            if dist < conexion_max_px and dist < min_dist:
                min_dist = dist
                punto_poligono = tuple(point)
                punto_amarillo = node
    
    if punto_poligono and punto_amarillo:
        G.add_edge(punto_poligono, punto_amarillo, weight=min_dist / scale_factor)
        poligono_vertices.append(punto_poligono)
    else:
        print(f"⚠️ Polígono {i+1} no conectado a la red amarilla")

# === CALCULAR RUTAS VÁLIDAS (SOLO POR AMARILLO) ===
with open("rutas_validas.csv", mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Poligono", "Distancia_ida_m", "Tiempo_ida_s", "Tiempo_regreso_s", "Tiempo_total_s"])
    
    for i, punto_poligono in enumerate(poligono_vertices):
        try:
            distancia = nx.shortest_path_length(G, source=punto_salida, target=punto_poligono, weight="weight")
            tiempo_ida = distancia / vel_ida
            tiempo_regreso = distancia / vel_regreso
            writer.writerow([
                i+1,
                round(distancia, 2),
                round(tiempo_ida, 1),
                round(tiempo_regreso, 1),
                round(tiempo_ida + tiempo_regreso, 1)
            ])
        except nx.NetworkXNoPath:
            print(f"🚨 No hay ruta válida al polígono {i+1}")
            writer.writerow([i+1, "N/A", "N/A", "N/A", "N/A"])

# === VISUALIZACIÓN ===
img_rutas = image.copy()
for punto_poligono in poligono_vertices:
    try:
        ruta = nx.shortest_path(G, source=punto_salida, target=punto_poligono, weight="weight")
        for k in range(len(ruta) - 1):
            cv2.line(img_rutas, ruta[k], ruta[k+1], (0, 0, 255), 2)
    except:
        continue

cv2.imwrite("rutas_validas.jpg", img_rutas)
print("✅ Proceso completado. Rutas válidas generadas.")

✅ Proceso completado. Rutas válidas generadas.


In [54]:
import cv2
import numpy as np
import networkx as nx
import csv
from math import sqrt

# ===== PARÁMETROS REALES =====
ESCALA_PX_A_METROS = 0.006 / 100  # 0.6 cm -> 100m (0.00006 m/px)
VELOCIDAD_IDA = 25 / 3.6  # 25 km/h -> m/s
VELOCIDAD_REGRESO = 40 / 3.6  # 40 km/h -> m/s
DISTANCIA_MAX_CONEXION = 4000  # píxeles máx para conectar nodos

# ===== PROCESAMIENTO DE IMAGEN =====
img = cv2.imread("Color.jpg")
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Detección de líneas amarillas (caminos obligatorios)
mask_amarillo = cv2.inRange(hsv, (20, 100, 100), (40, 255, 255))
kernel = np.ones((5,5), np.uint8)
mask_amarillo = cv2.morphologyEx(mask_amarillo, cv2.MORPH_CLOSE, kernel)  # Cierra huecos
contours_amarillo, _ = cv2.findContours(mask_amarillo, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Detección de almacén (verde)
mask_verde = cv2.inRange(hsv, (40, 100, 100), (80, 255, 255))
contours_verde, _ = cv2.findContours(mask_verde, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if not contours_verde:
    raise ValueError("No se encontró el almacén verde")

M = cv2.moments(contours_verde[0])
almacen = (int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"]))

# ===== CONSTRUCCIÓN DEL GRAFO =====
G = nx.Graph()

# 1. Añadir TODOS los puntos amarillos como nodos conectados
puntos_amarillos = []
for cnt in contours_amarillo:
    for punto in cnt[:,0,:]:
        puntos_amarillos.append(tuple(punto))

# Conectar puntos cercanos en líneas amarillas
for i, p1 in enumerate(puntos_amarillos):
    for p2 in puntos_amarillos[i+1:]:
        distancia_px = sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
        if distancia_px < 15:  # Conectar solo puntos cercanos en la misma línea
            G.add_edge(p1, p2, weight=distancia_px * ESCALA_PX_A_METROS)

# 2. Conectar almacén a la red amarilla
conectado = False
for p in puntos_amarillos:
    distancia_px = sqrt((almacen[0]-p[0])**2 + (almacen[1]-p[1])**2)
    if distancia_px < DISTANCIA_MAX_CONEXION:
        G.add_edge(almacen, p, weight=distancia_px * ESCALA_PX_A_METROS)
        conectado = True
if not conectado:
    raise ValueError("El almacén no está conectado a la red amarilla")

# 3. Conectar polígonos (magenta) a la red amarilla
mask_magenta = cv2.inRange(hsv, (140, 50, 50), (170, 255, 255))
contours_magenta, _ = cv2.findContours(mask_magenta, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

poligonos = []
for i, cnt in enumerate(contours_magenta):
    # Encontrar punto más cercano en el polígono a la red amarilla
    min_dist = float('inf')
    punto_poligono = None
    punto_amarillo = None
    
    for punto in cnt[:,0,:]:
        for p_amarillo in puntos_amarillos:
            dist = sqrt((punto[0]-p_amarillo[0])**2 + (punto[1]-p_amarillo[1])**2)
            if dist < DISTANCIA_MAX_CONEXION and dist < min_dist:
                min_dist = dist
                punto_poligono = tuple(punto)
                punto_amarillo = p_amarillo
    
    if punto_poligono and punto_amarillo:
        G.add_edge(punto_poligono, punto_amarillo, weight=min_dist * ESCALA_PX_A_METROS)
        poligonos.append((i+1, punto_poligono))
    else:
        print(f"⚠️ Polígono {i+1} no conectado")

# ===== CÁLCULO DE RUTAS =====
with open("rutas_correctas.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Polígono", "Distancia (m)", "Tiempo ida (s)", "Tiempo regreso (s)"])
    
    for poligono, punto in poligonos:
        try:
            distancia = nx.shortest_path_length(G, source=almacen, target=punto, weight="weight")
            tiempo_ida = distancia / VELOCIDAD_IDA
            tiempo_regreso = distancia / VELOCIDAD_REGRESO
            writer.writerow([
                poligono,
                round(distancia, 2),
                round(tiempo_ida, 1),
                round(tiempo_regreso, 1)
            ])
        except nx.NetworkXNoPath:
            print(f"🚨 No hay ruta al polígono {poligono}")
            writer.writerow([poligono, "N/A", "N/A", "N/A"])

# ===== VISUALIZACIÓN =====
img_rutas = img.copy()
for _, punto in poligonos:
    try:
        path = nx.shortest_path(G, source=almacen, target=punto, weight="weight")
        for i in range(len(path)-1):
            cv2.line(img_rutas, path[i], path[i+1], (0,0,255), 3)
    except:
        continue

cv2.imwrite("rutas_correctas.jpg", img_rutas)
print("✅ Análisis completado. Ver 'rutas_correctas.csv' y 'rutas_correctas.jpg'")

✅ Análisis completado. Ver 'rutas_correctas.csv' y 'rutas_correctas.jpg'


In [58]:
import cv2
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import pandas as pd
import csv

# ===== PARÁMETROS =====
ESCALA_PX_A_METROS = 100 / 0.006  # 100m reales = 0.6 cm en imagen
VELOCIDAD_IDA = 25 / 3.6  # m/s
VELOCIDAD_REGRESO = 40 / 3.6  # m/s

# ===== PROCESAMIENTO DE IMAGEN =====
img = cv2.imread("Color.jpg")
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 1. Detección del almacén (verde)
mask_verde = cv2.inRange(hsv, (40, 100, 100), (80, 255, 255))
contours_verde, _ = cv2.findContours(mask_verde, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if not contours_verde:
    raise ValueError("No se detectó el almacén verde")

M = cv2.moments(contours_verde[0])
almacen_pt = Point(int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

# 2. Detección de polígonos (magenta)
mask_magenta = cv2.inRange(hsv, (140, 50, 50), (170, 255, 255))
contours_magenta, _ = cv2.findContours(mask_magenta, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Crear GeoDataFrame para polígonos
poligonos_data = []
for i, cnt in enumerate(contours_magenta):
    poligonos_data.append({
        'ID': i+1,
        'geometry': Polygon(cnt[:, 0, :])
    })
poligonos_gdf = gpd.GeoDataFrame(poligonos_data, geometry='geometry')

# 3. Detección de rutas amarillas
mask_amarillo = cv2.inRange(hsv, (20, 100, 100), (40, 255, 255))
kernel = np.ones((5,5), np.uint8)
mask_amarillo = cv2.morphologyEx(mask_amarillo, cv2.MORPH_CLOSE, kernel)
contours_amarillo, _ = cv2.findContours(mask_amarillo, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Crear GeoDataFrame para rutas
rutas_data = []
for i, cnt in enumerate(contours_amarillo):
    if len(cnt) >= 2:  # Necesitamos al menos 2 puntos para una LineString
        rutas_data.append({
            'ID': i+1,
            'geometry': LineString(cnt[:, 0, :])
        })
rutas_gdf = gpd.GeoDataFrame(rutas_data, geometry='geometry')

# ===== CONEXIÓN DE COMPONENTES =====
# 1. Conectar almacén a la ruta más cercana
if not rutas_gdf.empty:
    almacen_gdf = gpd.GeoDataFrame([{'geometry': almacen_pt}])
    distancia = almacen_gdf.distance(rutas_gdf.unary_union)
    if distancia.min() * ESCALA_PX_A_METROS > 10:
        print("⚠️ Almacén lejos de rutas amarillas")

# 2. Conectar polígonos a rutas
poligonos_gdf['Distancia_Ruta'] = poligonos_gdf.geometry.distance(rutas_gdf.unary_union) * ESCALA_PX_A_METROS
poligonos_gdf['Conectado'] = poligonos_gdf['Distancia_Ruta'] <= 3500  # 15m máximo

# ===== CÁLCULO DE RUTAS =====
resultados = []
for _, poligono in poligonos_gdf.iterrows():
    if poligono['Conectado']:
        distancia = poligono.geometry.distance(rutas_gdf.unary_union) * ESCALA_PX_A_METROS
        tiempo_ida = distancia / VELOCIDAD_IDA
        tiempo_regreso = distancia / VELOCIDAD_REGRESO
        resultados.append([poligono['ID'], round(distancia, 2), round(tiempo_ida, 1), round(tiempo_regreso, 1)])
    else:
        resultados.append([poligono['ID'], "N/A", "N/A", "N/A"])

# ===== EXPORTAR RESULTADOS =====
with open("rutas_geometricas.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Polígono", "Distancia (m)", "Tiempo ida (s)", "Tiempo regreso (s)"])
    writer.writerows(resultados)

# Guardar capas GIS
poligonos_gdf.to_file("poligonos.shp")
rutas_gdf.to_file("rutas.shp")
Point(0,0).buffer(1).difference(Point(0,0).buffer(0.5))  # Crear archivo vacío para almacén
gpd.GeoDataFrame([{'geometry': almacen_pt}]).to_file("almacen.shp")

print("✅ Proceso completado. Resultados en rutas_geometricas.csv")

⚠️ Almacén lejos de rutas amarillas
✅ Proceso completado. Resultados en rutas_geometricas.csv


/var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/ipykernel_78507/2551024061.py:59: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  distancia = almacen_gdf.distance(rutas_gdf.unary_union)
/var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/ipykernel_78507/2551024061.py:64: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  poligonos_gdf['Distancia_Ruta'] = poligonos_gdf.geometry.distance(rutas_gdf.unary_union) * ESCALA_PX_A_METROS
/var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/ipykernel_78507/2551024061.py:85: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  poligonos_gdf.to_file("poligonos.shp")
/Users/samanthabritoozuna/anaconda3/envs/tecs6-arm/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other

In [60]:
import cv2
import numpy as np
import networkx as nx
import csv

# === PARÁMETROS ===
scale_factor = 100 / 0.006  # píxeles por metro (100 m / 0.6 cm)
vel_ida = 25 * 1000 / 3600  # m/s
vel_regreso = 40 * 1000 / 3600  # m/s
conexion_max_px = 3500  # máxima distancia en píxeles para conectar la camioneta

# === CARGAR IMAGEN Y CONVERTIR A HSV ===
image = cv2.imread("Color.jpg")
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# === DETECTAR CONTORNOS MAGENTA ===
lower_magenta = np.array([140, 50, 50])
upper_magenta = np.array([170, 255, 255])
mask_magenta = cv2.inRange(hsv, lower_magenta, upper_magenta)
contours, _ = cv2.findContours(mask_magenta, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# === DETECTAR ZONA VERDE (CAMIONETA) ===
lower_green = np.array([40, 100, 100])
upper_green = np.array([80, 255, 255])
mask_green = cv2.inRange(hsv, lower_green, upper_green)
contours_green, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if not contours_green:
    raise Exception("No se detectó la región verde (camioneta)")

M_green = cv2.moments(contours_green[0])
cx_green = int(M_green["m10"] / M_green["m00"])
cy_green = int(M_green["m01"] / M_green["m00"])
punto_salida = (cx_green, cy_green)

# === CONSTRUIR GRAFO DE CAMINOS (bordes magenta) ===
G = nx.Graph()
poligono_vertices = []  # Para guardar los vértices por polígono

for i, cnt in enumerate(contours):
    vertices = []
    for j in range(len(cnt)):
        p1 = tuple(cnt[j][0])
        p2 = tuple(cnt[(j + 1) % len(cnt)][0])
        dist = np.linalg.norm(np.array(p1) - np.array(p2))
        G.add_edge(p1, p2, weight=dist / scale_factor)
        vertices.append(p1)
    poligono_vertices.append(vertices)

# === CONECTAR CAMIONETA A TODOS LOS NODOS CERCANOS ===
all_nodes = list(G.nodes())
for node in all_nodes:
    dist = np.linalg.norm(np.array(punto_salida) - np.array(node))
    if dist < conexion_max_px:
        G.add_edge(punto_salida, node, weight=dist / scale_factor)

# === CALCULAR RUTAS DESDE ALMACÉN Y GUARDAR CSV ===
with open("rutas_desde_camioneta.csv", mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Poligono", "Distancia_ida_m", "Tiempo_ida_s", "Tiempo_regreso_s", "Tiempo_total_s"])

    for i, vertices in enumerate(poligono_vertices):
        min_dist = float("inf")
        nodo_objetivo = None
        for v in vertices:
            try:
                d = nx.shortest_path_length(G, source=punto_salida, target=v, weight="weight")
                if d < min_dist:
                    min_dist = d
                    nodo_objetivo = v
            except nx.NetworkXNoPath:
                continue
        if nodo_objetivo is None:
            print(f"⚠️ No se pudo encontrar ruta al polígono {i+1}")
            writer.writerow([i + 1, "N/A", "N/A", "N/A", "N/A"])
        else:
            tiempo_ida = min_dist / vel_ida
            tiempo_regreso = min_dist / vel_regreso
            writer.writerow([i + 1, round(min_dist, 2), round(tiempo_ida, 1), round(tiempo_regreso, 1), round(tiempo_ida + tiempo_regreso, 1)])

# === MATRIZ DE DISTANCIAS ENTRE POLÍGONOS ===
n = len(poligono_vertices)
matriz = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        if i == j:
            continue
        min_dist = float("inf")
        for v1 in poligono_vertices[i]:
            for v2 in poligono_vertices[j]:
                try:
                    d = nx.shortest_path_length(G, source=v1, target=v2, weight="weight")
                    if d < min_dist:
                        min_dist = d
                except nx.NetworkXNoPath:
                    continue
        if min_dist == float("inf"):
            matriz[i, j] = -1  # -1 para indicar que no hay camino
        else:
            matriz[i, j] = min_dist

# Guardar matriz en CSV
with open("matriz_distancias_poligonos.csv", "w", newline="") as f:
    writer = csv.writer(f)
    header = [""] + [f"P{j+1}" for j in range(n)]
    writer.writerow(header)
    for i in range(n):
        row = [f"P{i+1}"] + [round(matriz[i, j], 2) if matriz[i, j] >= 0 else "N/A" for j in range(n)]
        writer.writerow(row)

# === VISUALIZAR LAS RUTAS ===
img_rutas = image.copy()
for i, vertices in enumerate(poligono_vertices):
    min_dist = float("inf")
    nodo_objetivo = None
    for v in vertices:
        try:
            d = nx.shortest_path_length(G, source=punto_salida, target=v, weight="weight")
            if d < min_dist:
                min_dist = d
                nodo_objetivo = v
        except nx.NetworkXNoPath:
            continue
    if nodo_objetivo:
        ruta = nx.shortest_path(G, source=punto_salida, target=nodo_objetivo, weight="weight")
        for k in range(len(ruta) - 1):
            cv2.line(img_rutas, ruta[k], ruta[k+1], (0, 0, 255), 2)

cv2.imwrite("rutas_camioneta.jpg", img_rutas)


KeyboardInterrupt: 

# Algoritmo

## Librerías

In [8]:
import math
import pandas as pd


## Datasets

In [15]:

poligonos_df = pd.DataFrame({
    "poligono": list(range(1, 32)),
    "hectareas": [5.4, 7.52, 8, 8, 7.56, 4.19, 6.28, 7.6, 8, 8, 7.67, 1.47, 7.97,
                  5.98, 5.4, 5.64, 6.11, 7.11, 4.92, 1.38, 8, 7.82, 5.53, 5.64,
                  5.05, 4.75, 1.28, 6.64, 6.54, 6.76, 7.34],
    "plantas_total": [1107, 1580, 1860, 1769, 1556, 928, 1360, 1585, 1860, 1769,
                      1770, 317, 1784, 1451, 1196, 1198, 1300, 1635, 1018, 213,
                      1769, 1713, 1280, 1258, 1099, 1068, 178, 1479, 1444, 1680, 1451]
})


In [16]:
poligonos_df

,poligono,hectareas,plantas_total
0,1,5.40,1107
1,2,7.52,1580
2,3,8.00,1860
3,4,8.00,1769
4,5,7.56,1556
5,6,4.19,928
6,7,6.28,1360
7,8,7.60,1585
8,9,8.00,1860
9,10,8.00,1769


In [17]:
especies_df = pd.DataFrame({
    "especie": [
        "Agave lechuguilla", "Agave salmiana", "Agave scabra", "Agave striata",
        "Opuntia cantabrigiensis", "Opuntia engelmani", "Opuntia robusta",
        "Opuntia streptacanta", "Prosopis laevigata", "Yucca filifera"
    ],
    "porcentaje": [
        6.2977, 29.9618, 6.2977, 6.2977, 7.4427, 5.7252,
        11.0687, 9.7328, 13.1679, 4.0076
    ],
    "altura_cm": [
        40, 40, 20, 25, 25, 25, 25, 27.5, 27.5, 25
    ]
})


In [18]:
especies_df["volumen_m3"] = (0.25 * 0.25) * (especies_df["altura_cm"] / 100)


In [19]:
especies_df

,especie,porcentaje,altura_cm,volumen_m3
0,Agave lechuguilla,6.2977,40.0,0.025000
1,Agave salmiana,29.9618,40.0,0.025000
2,Agave scabra,6.2977,20.0,0.012500
3,Agave striata,6.2977,25.0,0.015625
4,Opuntia cantabrigiensis,7.4427,25.0,0.015625
5,Opuntia engelmani,5.7252,25.0,0.015625
6,Opuntia robusta,11.0687,25.0,0.015625
7,Opuntia streptacanta,9.7328,27.5,0.017188
8,Prosopis laevigata,13.1679,27.5,0.017188
9,Yucca filifera,4.0076,25.0,0.015625


In [9]:
# Parámetros de la camioneta
capacidad_camioneta = 1.24  # m³
tiempo_por_viaje = 90  # minutos (carga + descarga + traslado)
tiempo_jornada = 360  # minutos (6 horas)


In [10]:
# Inicialización
viajes_dia = []
dia_actual = 1
ha_en_dia = 0
plantas_en_dia = 0

In [11]:
# Parámetros de almacenamiento y tratamiento
capacidad_almacenamiento_m2 = 400  # m²
capacidad_tratamiento_m2 = 100  # m²
tamano_planta_m2 = 0.25 * 0.25  # m² (25 cm x 25 cm por planta)

## Test

In [12]:
#Función para calcular las plantas a enviar desde vivero a almacenamiento
def calcular_plantas_a_enviar(porcentaje, plantas_total):
    return porcentaje * plantas_total

# Inicialización
viajes_dia = []
dia_actual = 1
ha_en_dia = 0
plantas_en_dia = 0
almacenamiento_disponible = capacidad_almacenamiento_m2  # m² disponibles para almacenamiento

# Distribución parcial de plantación
for idx, row in poligonos_df.iterrows():
    poligono = row["poligono"]
    total_plantas = row["plantas_total"]
    ha = row["hectareas"]
    
    # Determinar cuántas hectáreas plantar por día
    hectares_por_dia = min(ha, 5 - ha_en_dia)  # No exceder 5 ha/día
    if hectares_por_dia == ha:
        print(f"Completando {ha} ha en el polígono {poligono}.")
    else:
        print(f"Plantando {hectares_por_dia} ha de {ha} ha en el polígono {poligono}.")
    
    # Calcular número de plantas a plantar
    plantas_a_plantar = (hectares_por_dia / ha) * total_plantas

    # Calcular volumen total necesario para el polígono a plantar
    total_vol = 0
    for _, esp in especies_df.iterrows():
        porcentaje = esp["porcentaje"] / 100
        n_plantas = calcular_plantas_a_enviar(porcentaje, plantas_a_plantar)
        vol_total = n_plantas * esp["volumen_m3"]
        total_vol += vol_total
    
    viajes_necesarios = math.ceil(total_vol / capacidad_camioneta)

    # Verificar almacenamiento disponible
    plantas_a_almacenar = plantas_a_plantar
    espacio_requerido = plantas_a_almacenar * tamano_planta_m2

    if espacio_requerido > almacenamiento_disponible:
        print(f"⚠️ El almacenamiento no es suficiente para el polígono {poligono}. Requiere más espacio.")
        # Aquí se debe gestionar que las plantas se distribuyan en días
        # posteriores o en almacenamiento adicional si es posible
        continue

    # Calcular tiempo requerido para la plantación en ese día
    tiempo_total = viajes_necesarios * tiempo_por_viaje

    # Verificar si se excede la jornada
    if tiempo_total > tiempo_jornada:
        print(f"⚠️ El polígono {poligono} no cabe en este día. Se distribuirá en días posteriores.")
        dias_requeridos = math.ceil(tiempo_total / tiempo_jornada)
        for dia in range(dia_actual, dia_actual + dias_requeridos):
            viajes_dia.append({
                "dia": dia,
                "poligono": poligono,
                "ha": hectares_por_dia,
                "plantas": plantas_a_plantar,
                "viajes": viajes_necesarios,
                "tiempo_min": tiempo_total
            })
        dia_actual += dias_requeridos
    else:
        viajes_dia.append({
            "dia": dia_actual,
            "poligono": poligono,
            "ha": hectares_por_dia,
            "plantas": plantas_a_plantar,
            "viajes": viajes_necesarios,
            "tiempo_min": tiempo_total
        })
    
    ha_en_dia += hectares_por_dia
    plantas_en_dia += plantas_a_plantar

    # Si se alcanzan las 5 ha por día, iniciar un nuevo día
    if ha_en_dia >= 5 or tiempo_total >= tiempo_jornada:
        dia_actual += 1
        ha_en_dia = 0
        plantas_en_dia = 0

# Resultado: Planificación de plantación por día
plantacion_planificada = pd.DataFrame(viajes_dia)
print(plantacion_planificada)

Plantando 5 ha de 5.4 ha en el polígono 1.0.
⚠️ El polígono 1.0 no cabe en este día. Se distribuirá en días posteriores.
Plantando 5 ha de 7.52 ha en el polígono 2.0.
⚠️ El polígono 2.0 no cabe en este día. Se distribuirá en días posteriores.
Plantando 5 ha de 8.0 ha en el polígono 3.0.
⚠️ El polígono 3.0 no cabe en este día. Se distribuirá en días posteriores.
Plantando 5 ha de 8.0 ha en el polígono 4.0.
⚠️ El polígono 4.0 no cabe en este día. Se distribuirá en días posteriores.
Plantando 5 ha de 7.56 ha en el polígono 5.0.
⚠️ El polígono 5.0 no cabe en este día. Se distribuirá en días posteriores.
Completando 4.19 ha en el polígono 6.0.
⚠️ El polígono 6.0 no cabe en este día. Se distribuirá en días posteriores.
Plantando 5 ha de 6.28 ha en el polígono 7.0.
⚠️ El polígono 7.0 no cabe en este día. Se distribuirá en días posteriores.
Plantando 5 ha de 7.6 ha en el polígono 8.0.
⚠️ El polígono 8.0 no cabe en este día. Se distribuirá en días posteriores.
Plantando 5 ha de 8.0 ha en el pol

In [13]:
import math
import pandas as pd

# Datos de Polígonos
poligonos_df = pd.DataFrame({
    "poligono": list(range(1, 32)),
    "hectareas": [5.4, 7.52, 8, 8, 7.56, 4.19, 6.28, 7.6, 8, 8, 7.67, 1.47, 7.97,
                  5.98, 5.4, 5.64, 6.11, 7.11, 4.92, 1.38, 8, 7.82, 5.53, 5.64,
                  5.05, 4.75, 1.28, 6.64, 6.54, 6.76, 7.34],
    "plantas_total": [1107, 1580, 1860, 1769, 1556, 928, 1360, 1585, 1860, 1769,
                      1770, 317, 1784, 1451, 1196, 1198, 1300, 1635, 1018, 213,
                      1769, 1713, 1280, 1258, 1099, 1068, 178, 1479, 1444, 1680, 1451]
})

# Datos de Especies
especies_df = pd.DataFrame({
    "especie": [
        "Agave lechuguilla", "Agave salmiana", "Agave scabra", "Agave striata",
        "Opuntia cantabrigiensis", "Opuntia engelmani", "Opuntia robusta",
        "Opuntia streptacanta", "Prosopis laevigata", "Yucca filifera"
    ],
    "porcentaje": [
        6.2977, 29.9618, 6.2977, 6.2977, 7.4427, 5.7252,
        11.0687, 9.7328, 13.1679, 4.0076
    ],
    "altura_cm": [
        40, 40, 20, 25, 25, 25, 25, 27.5, 27.5, 25
    ],
    "volumen_m3": [0.25 * 0.25 * (h / 100) for h in [40, 40, 20, 25, 25, 25, 25, 27.5, 27.5, 25]]
})

# Parámetros de la camioneta
capacidad_camioneta = 1.24  # m³
tiempo_por_viaje = 90  # minutos (carga + descarga + traslado)
tiempo_jornada = 360  # minutos (6 horas)

# Parámetros de almacenamiento y tratamiento
capacidad_almacenamiento_m2 = 400  # m²
capacidad_tratamiento_m2 = 100  # m²
tamano_planta_m2 = 0.25 * 0.25  # m² (25 cm x 25 cm por planta)

# Función para calcular las plantas a enviar desde vivero a almacenamiento
def calcular_plantas_a_enviar(porcentaje, plantas_total):
    return porcentaje * plantas_total

# Inicialización
viajes_dia = []
dia_actual = 1
ha_en_dia = 0
plantas_en_dia = 0
almacenamiento_disponible = capacidad_almacenamiento_m2  # m² disponibles para almacenamiento

# Selección flexible de hectáreas para cumplir 5 hectáreas por día
while dia_actual <= len(poligonos_df):
    ha_requerida = 5
    hectareas_plantadas = 0
    plantas_en_dia = 0
    viajes_necesarios_dia = 0

    # Selección de polígonos para completar las 5 hectáreas
    seleccion_poligonos = []
    for idx, row in poligonos_df.iterrows():
        if hectareas_plantadas < ha_requerida:
            hectareas_a_plantar = min(row['hectareas'], ha_requerida - hectareas_plantadas)
            hectareas_plantadas += hectareas_a_plantar
            plantas_a_plantar = (hectareas_a_plantar / row['hectareas']) * row['plantas_total']
            seleccion_poligonos.append({
                'poligono': row['poligono'],
                'hectareas': hectareas_a_plantar,
                'plantas': plantas_a_plantar
            })
            plantas_en_dia += plantas_a_plantar

    # Calcular volumen total necesario para el día
    total_vol = 0
    for esp_idx, esp_row in especies_df.iterrows():
        porcentaje = esp_row["porcentaje"] / 100
        for pol in seleccion_poligonos:
            plantas_a_plantar = (porcentaje * pol['plantas'])
            vol_total = plantas_a_plantar * esp_row["volumen_m3"]
            total_vol += vol_total

    # Calcular viajes necesarios para transportar el volumen de plantas
    viajes_necesarios_dia = math.ceil(total_vol / capacidad_camioneta)

    # Verificar espacio de almacenamiento
    espacio_requerido = plantas_en_dia * tamano_planta_m2
    if espacio_requerido > almacenamiento_disponible:
        print(f"⚠️ El almacenamiento no es suficiente para el día {dia_actual}. Requiere más espacio.")
        break  # Aquí puedes dividir y continuar en días posteriores

    # Calcular tiempo requerido para el día
    tiempo_total = viajes_necesarios_dia * tiempo_por_viaje

    # Verificar si se excede la jornada
    if tiempo_total > tiempo_jornada:
        print(f"⚠️ El tiempo de plantación excede la jornada laboral en el día {dia_actual}. Requiere más tiempo.")
        break  # Aquí puedes dividir el trabajo en más días

    # Registrar viajes de plantas
    for pol in seleccion_poligonos:
        viajes_dia.append({
            "dia": dia_actual,
            "poligono": pol['poligono'],
            "hectareas": pol['hectareas'],
            "plantas": pol['plantas'],
            "viajes": viajes_necesarios_dia,
            "tiempo_min": tiempo_total
        })
    
    # Actualizar jornada
    dia_actual += 1
    ha_en_dia = 0

# Resultado: Planificación de plantación por día
plantacion_planificada = pd.DataFrame(viajes_dia)
print(plantacion_planificada)


⚠️ El tiempo de plantación excede la jornada laboral en el día 1. Requiere más tiempo.
Empty DataFrame
Columns: []
Index: []


In [ ]:



# Distribución parcial de plantación
for idx, row in poligonos_df.iterrows():
    poligono = row["poligono"]
    total_plantas = row["plantas_total"]
    ha = row["hectareas"]
    
    # Determinar cuántas hectáreas plantar por día
    hectares_por_dia = min(ha, 5 - ha_en_dia)  # No exceder 5 ha/día
    if hectares_por_dia == ha:
        print(f"Completando {ha} ha en el polígono {poligono}.")
    else:
        print(f"Plantando {hectares_por_dia} ha de {ha} ha en el polígono {poligono}.")
    
    # Calcular número de plantas a plantar
    plantas_a_plantar = (hectares_por_dia / ha) * total_plantas

    # Calcular volumen total necesario para el polígono a plantar
    total_vol = 0
    for _, esp in especies_df.iterrows():
        porcentaje = esp["porcentaje"] / 100
        n_plantas = porcentaje * plantas_a_plantar
        vol_total = n_plantas * esp["volumen_m3"]
        total_vol += vol_total
    
    viajes_necesarios = math.ceil(total_vol / capacidad_camioneta)

    # Calcular tiempo requerido para la plantación en ese día
    tiempo_total = viajes_necesarios * tiempo_por_viaje

    # Verificar si se excede la jornada
    if tiempo_total > tiempo_jornada:
        print(f"⚠️ El polígono {poligono} no cabe en este día. Se distribuirá en días posteriores.")
        # Aquí se divide en días posteriores
        dias_requeridos = math.ceil(tiempo_total / tiempo_jornada)
        for dia in range(dia_actual, dia_actual + dias_requeridos):
            viajes_dia.append({
                "dia": dia,
                "poligono": poligono,
                "ha": hectares_por_dia,
                "plantas": plantas_a_plantar,
                "viajes": viajes_necesarios,
                "tiempo_min": tiempo_total
            })
        dia_actual += dias_requeridos
    else:
        viajes_dia.append({
            "dia": dia_actual,
            "poligono": poligono,
            "ha": hectares_por_dia,
            "plantas": plantas_a_plantar,
            "viajes": viajes_necesarios,
            "tiempo_min": tiempo_total
        })
    
    ha_en_dia += hectares_por_dia
    plantas_en_dia += plantas_a_plantar

    # Si se alcanzan las 5 ha por día, iniciar un nuevo día
    if ha_en_dia >= 5 or tiempo_total >= tiempo_jornada:
        dia_actual += 1
        ha_en_dia = 0
        plantas_en_dia = 0

# Resultado: Planificación de plantación por día
plantacion_planificada = pd.DataFrame(viajes_dia)
print(plantacion_planificada)

Plantando 5 ha de 5.4 ha en el polígono 1.0.
⚠️ El polígono 1.0 no cabe en este día. Se distribuirá en días posteriores.
Plantando 5 ha de 7.52 ha en el polígono 2.0.
⚠️ El polígono 2.0 no cabe en este día. Se distribuirá en días posteriores.
Plantando 5 ha de 8.0 ha en el polígono 3.0.
⚠️ El polígono 3.0 no cabe en este día. Se distribuirá en días posteriores.
Plantando 5 ha de 8.0 ha en el polígono 4.0.
⚠️ El polígono 4.0 no cabe en este día. Se distribuirá en días posteriores.
Plantando 5 ha de 7.56 ha en el polígono 5.0.
⚠️ El polígono 5.0 no cabe en este día. Se distribuirá en días posteriores.
Completando 4.19 ha en el polígono 6.0.
⚠️ El polígono 6.0 no cabe en este día. Se distribuirá en días posteriores.
Plantando 5 ha de 6.28 ha en el polígono 7.0.
⚠️ El polígono 7.0 no cabe en este día. Se distribuirá en días posteriores.
Plantando 5 ha de 7.6 ha en el polígono 8.0.
⚠️ El polígono 8.0 no cabe en este día. Se distribuirá en días posteriores.
Plantando 5 ha de 8.0 ha en el pol

In [ ]:
import pandas as pd

# Datos de planificación de plantación por día
plantacion_planificada = pd.DataFrame([
    {"dia": 1, "poligono": 1, "hectareas": 5.4, "plantas": 1107, "viajes": 1, "tiempo_min": 90},
    {"dia": 2, "poligono": 2, "hectareas": 5.0, "plantas": 1050, "viajes": 1, "tiempo_min": 90},
    {"dia": 2, "poligono": 3, "hectareas": 0.4, "plantas": 93, "viajes": 1, "tiempo_min": 90},
    {"dia": 3, "poligono": 3, "hectareas": 7.6, "plantas": 1767, "viajes": 2, "tiempo_min": 180},
    {"dia": 4, "poligono": 4, "hectareas": 5.0, "plantas": 1105, "viajes": 1, "tiempo_min": 90},
    {"dia": 5, "poligono": 4, "hectareas": 3.0, "plantas": 664, "viajes": 1, "tiempo_min": 90},
    {"dia": 5, "poligono": 5, "hectareas": 2.0, "plantas": 412, "viajes": 1, "tiempo_min": 90},
])

# Guardar en archivo Excel
output_path = "/mnt/data/planificacion_plantacion.xlsx"
plantacion_planificada.to_excel(output_path, index=False)

output_path
